In [1]:
!pip install palmerpenguins

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [29]:
from palmerpenguins import load_penguins
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Palmer Penguins Modeling

Import the Palmer Penguins dataset and print out the first few rows.

Suppose we want to predict `bill_depth_mm` using the other variables in the dataset.

**Dummify** all variables that require this.

In [16]:
penguins = load_penguins()
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [15]:
ct = ColumnTransformer(
    [("dummify", OneHotEncoder(sparse_output= False, drop = "first"), make_column_selector(dtype_include=object))]
).set_output(transform = "pandas")

penguins_dummified = ct.fit_transform(penguins)
penguins_dummified.head()

,dummify__species_Chinstrap,dummify__species_Gentoo,dummify__island_Dream,dummify__island_Torgersen,dummify__sex_male,dummify__sex_nan
0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0


Let's use the other variables to predict `bill_depth_mm`. Prepare your data and fit the following models on the entire dataset:

* Your best multiple linear regression model from before
* Two kNN models (for different values of K)
* A decision tree model

Create a plot like the right plot of Fig 1. in our `Model Validation` chapter with the training and test error plotted for each of your four models.

Which of your models was best?

In [14]:
#best linear regression model:

X = penguins_dummified.drop(columns = "bill_depth_mm")
y = penguins_dummified["bill_depth_mm"]




KeyError: "['bill_depth_mm'] not found in axis"

In [25]:
penguins = penguins.dropna()

In [26]:
X = penguins.drop(columns = "bill_depth_mm")
y = penguins["bill_depth_mm"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [31]:
#knn models:
ct_ = ColumnTransformer(
    [("scale", StandardScaler(), make_column_selector(dtype_include=np.number)),
    ("dummify", OneHotEncoder(sparse_output= False, drop = "first"), make_column_selector(dtype_include= object))]
).set_output(transform = "pandas")

knn_pipeline = Pipeline(
    [("pre_processing", ct_),
     ("knn", KNeighborsRegressor(n_neighbors = 3))
     ]).set_output(transform = "pandas")

knn_fitted = knn_pipeline.fit(X_train, y_train)
y_preds_test_knn = knn_fitted.predict(X_test)
y_preds_train_knn = knn_fitted.predict(X_train)

mse_test_knn = mean_squared_error(y_test, y_preds_test_knn)
mse_train_knn = mean_squared_error(y_train, y_preds_train_knn)
print(mse_test_knn, mse_train_knn)

0.7208201058201057 0.3579785809906291


In [32]:
knn_pipeline_2 = Pipeline(
    [("pre_processing", ct_),
     ("knn", KNeighborsRegressor(n_neighbors = 6))
     ]).set_output(transform = "pandas")

knn2_fitted = knn_pipeline_2.fit(X_train, y_train)
y_preds_test_knn2 = knn2_fitted.predict(X_test)
y_preds_train_knn2 = knn2_fitted.predict(X_train)

mse_test_knn2 = mean_squared_error(y_test, y_preds_test_knn2)
mse_train_knn2 = mean_squared_error(y_train, y_preds_train_knn2)
print(mse_test_knn2, mse_train_knn2)

0.7077711640211637 0.4581637661758143


In [34]:
#decision tree model

decision_tree_pipeline = Pipeline(
    [("pre_processing", ct_),
     ("decision_tree", DecisionTreeRegressor(min_samples_leaf = 5))
     ]).set_output(transform = "pandas")

decision_tree_fitted = decision_tree_pipeline.fit(X_train, y_train)
y_preds_test_dt = decision_tree_fitted.predict(X_test)
y_preds_train_dt = decision_tree_fitted.predict(X_train)
mse_test_dt = mean_squared_error(y_test, y_preds_test_dt)
mse_train_dt = mean_squared_error(y_train, y_preds_train_dt)
print(mse_test_dt, mse_train_dt)


0.9537657348356015 0.29638633900682093
